# Load libraries

In [3]:
import pandas as pd
import zipfile

# unzip compressed data

In [4]:
path_to_zip = './data/ml-25m.zip'
with zipfile.ZipFile(path_to_zip,"r") as zip_ref:
    zip_ref.extractall("./data")

In [7]:
data_sources = {
#     'imdb_basics'   : './data/name.basics.tsv.gz',
#     'imdb_ratings'  : './data/title.ratings.tsv.gz',
    'genome_scores' : './data/ml-25m/genome-scores.csv',
    'genome_tags'   : './data/ml-25m/genome-tags.csv',
    'links'         : './data/ml-25m/links.csv',
    'movies'        : './data/ml-25m/movies.csv',
    'ratings'       : './data/ml-25m/ratings.csv',
    'tags'          : './data/ml-25m/tags.csv',
}

In [8]:
dfs = {}
for name, path in data_sources.items():
    dfs[name] = pd.read_csv(path)

In [9]:
dfs.keys()

dict_keys(['genome_scores', 'genome_tags', 'links', 'movies', 'ratings', 'tags'])

In [26]:
all_ones = dfs['genome_scores'][dfs['genome_scores'].relevance==1]

In [25]:
combined_df = pd.merge(dfs['genome_scores'], dfs['genome_tags'], on='tagId')
combined_df.tag.value_counts()

prejudice          13816
depp & burton      13816
alcatraz           13816
small town         13816
book was better    13816
                   ...  
studio ghibli      13816
family bonds       13816
cia                13816
treasure hunt      13816
crime              13816
Name: tag, Length: 1128, dtype: int64

In [28]:
combined_df[combined_df.relevance == 1].tag.unique()

array(['007 (series)', 'aardman studios', 'adapted from:game',
       'afi 100 (movie quotes)', 'amy smart', 'android(s)/cyborg(s)',
       'author:alan moore', 'author:neil gaiman', 'bdsm', 'coen bros',
       'crime gone awry', 'dynamic cgi action', 'finnish', 'genocide',
       'ghosts/afterlife', 'goretastic', 'gypsy accent', 'hitchcock',
       'kick-butt women', 'martial arts', 'mother-son relationship',
       'ninja', 'notable nudity', 'nudity (rear)', 'nudity (topless)',
       'oscar (best effects - visual effects)',
       'oscar (best music - original score)',
       'oscar (best music - original song)', 'potential oscar nom',
       'saturn award (best science fiction film)',
       'saturn award (best special effects)', 'scifi cult',
       'single father', 'southern theme', 'spelling bee', 'time loop',
       'tom clancy', 'transgender', 'unusual plot structure',
       'vampire human love', 'view askew', 'wuxia'], dtype=object)

In [20]:
for name in data_sources:
    print(name)
    print(dfs[name].shape)
    print(dfs[name].head())    

genome_scores
(15584448, 3)
   movieId  tagId  relevance
0        1      1    0.02875
1        1      2    0.02375
2        1      3    0.06250
3        1      4    0.07575
4        1      5    0.14075
genome_tags
(1128, 2)
   tagId           tag
0      1           007
1      2  007 (series)
2      3  18th century
3      4         1920s
4      5         1930s
links
(62423, 3)
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0
movies
(62423, 3)
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Childr